In [ ]:
%cd /home/aditya/Documents/SolarCycleProject/code
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import gaussian
import src.preprocesser as pre
import src.network as network
import src.plotter as plotter 
from src.hyperparams import *
%load_ext autoreload

In [ ]:
%autoreload

In [ ]:
# Getting the data
headers = ["Year",
           "Month",
           "Day",
           "Decimal Date",
           "Daily Total Sunspot Number",
           "Sunspot Number Stddev",
           "No of observations",
           "Definitive/Provisional"
]
filename = "data/SN_d_tot_V2.0.csv"
data = pd.read_csv(filename, delimiter=";", names=headers)[:1000]

In [ ]:
data

In [ ]:
# Data Preprocessing
dates, spots, inverter = pre.preprocess(data)

In [ ]:
if mean_type == "gaussian":
    weights = gaussian(M=mean_length, std=0.1, sym=True)
    weights /= np.sum(weights) # normalise the weights
    spots = pre.running_mean_helper(spots, weights)
elif mean_type == "uniform":
    weights = np.ones(mean_length)/mean_length
    spots = pre.running_mean_helper(spots, weights)
else:
    pass


In [ ]:
X = spots
index = dates
x_slid, y_slid, idx_slid = pre.sliding_window_main(X, X, index)
x_train, y_train, idx_train, x_val, y_val, idx_val, x_test, y_test, idx_test = pre.data_splitting_main(x_slid, y_slid, idx_slid)

In [ ]:
hlayer_sizes = np.arange(1, 101, 1)
nets = []
for layer_size in hlayer_sizes:
    net = network.create_network(layer_size=layer_size)
    nets.append(net)

In [ ]:
histories = []
losses = []
val_losses = []
for i in range(len(hlayer_sizes)):
    net = nets[i]
    print("Training network with hidden layer size:", hlayer_sizes[i])
    history = network.trainer(net, x_train, y_train, x_val, y_val, verbose=0)
    loss = history.history["loss"][-1]
    losses.append(loss)
    val_loss = history.history["val_loss"][-1]
    val_losses.append(val_loss)
    print("epochs: %d - loss: %.4f - val_loss: %.4f" % (epochs, loss, val_loss))
    print("Training Completed")
    print()
    histories.append(history)
    
var_train = np.var(y_train)
var_val = np.var(y_val)
print("Variance in y_train:", var_train)
print("Variance in y_val:", var_val)

In [ ]:
for i in range(len(hlayer_sizes)):
    net = nets[i]
    history = histories[i]
    plotter.plot_predictions(net, x_train, y_train, idx_train, x_val, y_val, idx_val)
    plt.close()
    plotter.plot_loss_vs_epoch(history, var_train, var_val, show=False)
    plt.close()

In [ ]:
plt.plot(hlayer_sizes, losses, label="Training Loss", marker="o")
plt.plot(hlayer_sizes, val_losses, label="Validation Loss", marker="o")
plt.xlabel("Hidden Layer Size")
plt.ylabel("Mean Square Loss")
plt.legend()
plt.savefig("img/loss_vs_layersize.png")